# Arbitron database connectivity test

to reset Jupyter theme type "jt -r" in command line
(see https://github.com/dunovank/jupyter-themes)

In [48]:
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime
from markets.markets import Markets
from database.database import Database
from settings.settings import Settings

In [3]:
db = Database()

Connecting to database...OK


In [4]:
db.get_exchanges()
db.get_tokens()
db.get_pairs()

In [5]:
df_exchangs = db.df_exchanges
df_tokens = db.df_tokens
df_pairs = db.df_pairs[['pair', 'exchange']]

In [6]:
markets = Markets(db)

CCXT version: 1.15.27


In [7]:
#df_pairs.sort_values(['pair','exchange']).set_index('pair')
pairs = pd.DataFrame(df_pairs.groupby(['pair'])['exchange'].apply(list)).exchange.to_dict()

In [7]:
pairs

{'ADA/BTC': ['binance', 'bittrex', 'hitbtc2', 'cryptopia'],
 'ADA/DOGE': ['cryptopia'],
 'ADA/ETH': ['binance', 'bittrex', 'hitbtc2'],
 'ADA/LTC': ['cryptopia'],
 'ADA/USDT': ['binance', 'bittrex', 'hitbtc2'],
 'BCH/BTC': ['kraken',
  'poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BCH/DOGE': ['yobit', 'cryptopia'],
 'BCH/ETH': ['poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2'],
 'BCH/LTC': ['cryptopia'],
 'BCH/USD': ['kraken', 'yobit', 'okex', 'exmo'],
 'BCH/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'hitbtc2',
  'cryptopia'],
 'BTC/USD': ['kraken', 'yobit', 'okex', 'bittrex', 'exmo'],
 'BTC/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BTG/BTC': ['yobit', 'binance', 'okex', 'bittrex', 'exmo', 'hitbtc2'],
 'BTG/DOGE': ['yobit'],
 'BTG/ETH': ['yobit', 'binance', 'bittrex', 'hitbtc2'],
 'BTG/USD': ['yobit', 'okex', 'exmo'],
 'BTG/USDT': ['

In [8]:
exchanges = df_pairs.groupby(['exchange'])['pair'].apply(list).to_dict()

In [9]:
exchanges

{'binance': ['ADA/BTC',
  'ADA/ETH',
  'ADA/USDT',
  'BCH/BTC',
  'BCH/ETH',
  'BCH/USDT',
  'BTC/USDT',
  'BTG/BTC',
  'BTG/ETH',
  'DASH/BTC',
  'DASH/ETH',
  'EOS/BTC',
  'EOS/ETH',
  'EOS/USDT',
  'ETC/BTC',
  'ETC/ETH',
  'ETC/USDT',
  'ETH/BTC',
  'ETH/USDT',
  'ICX/BTC',
  'ICX/ETH',
  'ICX/USDT',
  'LTC/BTC',
  'LTC/ETH',
  'LTC/USDT',
  'NEO/BTC',
  'NEO/ETH',
  'NEO/USDT',
  'OMG/BTC',
  'OMG/ETH',
  'SC/BTC',
  'SC/ETH',
  'TRX/BTC',
  'TRX/ETH',
  'TRX/USDT',
  'XEM/BTC',
  'XEM/ETH',
  'XLM/BTC',
  'XLM/ETH',
  'XLM/USDT',
  'XMR/BTC',
  'XMR/ETH',
  'XRP/BTC',
  'XRP/ETH',
  'XRP/USDT',
  'XVG/BTC',
  'XVG/ETH',
  'ZEC/BTC',
  'ZEC/ETH'],
 'bittrex': ['ADA/BTC',
  'ADA/ETH',
  'ADA/USDT',
  'BCH/BTC',
  'BCH/ETH',
  'BCH/USDT',
  'BTC/USD',
  'BTC/USDT',
  'BTG/BTC',
  'BTG/ETH',
  'BTG/USDT',
  'DASH/BTC',
  'DASH/ETH',
  'DASH/USDT',
  'DCR/BTC',
  'DCR/USDT',
  'DOGE/BTC',
  'ETC/BTC',
  'ETC/ETH',
  'ETC/USDT',
  'ETH/BTC',
  'ETH/USD',
  'ETH/USDT',
  'LTC/BTC',
  'L

In [10]:
exchanges['hitbtc2'][3:5]

['BCH/BTC', 'BCH/ETH']

In [11]:
print(exchanges['exmo'])

['BCH/BTC', 'BCH/ETH', 'BCH/USD', 'BTC/USD', 'BTC/USDT', 'BTG/BTC', 'BTG/USD', 'DASH/BTC', 'DASH/USD', 'DOGE/BTC', 'EOS/BTC', 'EOS/USD', 'ETC/BTC', 'ETC/USD', 'ETH/BTC', 'ETH/LTC', 'ETH/USD', 'ETH/USDT', 'LTC/BTC', 'LTC/USD', 'USDT/USD', 'XLM/BTC', 'XLM/USD', 'XMR/BTC', 'XMR/USD', 'XRP/BTC', 'XRP/USD', 'ZEC/BTC', 'ZEC/USD']


# Get last access time

In [14]:
exchange = "exmo"
pair = "BCH/BTC"
cql = f"""SELECT ts FROM arbitron.history WHERE exchange='{exchange}' and pair='{pair}' LIMIT 1"""
cql

"SELECT ts FROM arbitron.history WHERE exchange='exmo' and pair='BCH/BTC' LIMIT 1"

In [17]:
res = db.session.execute(cql, timeout=5)
df = res._current_rows

In [52]:
int(df.iloc[0].values[0].astype(np.uint64)/1e6)

1532592654000

## Cool C* PER PARTITION feature for fetching last access times in single pass

In [53]:
cql = "select * from arbitron.history per partition limit 1"

In [54]:
res = db.session.execute(cql, timeout=5)
df = res._current_rows

In [102]:
df.sort_values(['exchange','pair'], inplace=True)
df.head(20) # ts contains last timestamps

,exchange,pair,ts,id,amount,insert_date,price,side,type
83,binance,ADA/BTC,2018-07-26 14:17:53.964,12838810,486.00000,2018-07-26 14:18:00.843,0.000021,buy,None
125,binance,ADA/ETH,2018-07-26 14:17:50.734,5419062,479.00000,2018-07-26 14:18:01.856,0.000360,sell,None
219,binance,ADA/USDT,2018-07-26 14:18:01.744,3828066,399.30000,2018-07-26 14:18:03.323,0.171500,buy,None
258,binance,BCH/BTC,2018-07-26 14:18:04.607,10261780,0.00500,2018-07-26 14:18:04.804,0.101682,sell,None
176,binance,BCH/ETH,2018-07-26 14:16:37.594,1711307,1.41700,2018-07-26 14:17:13.885,1.757000,sell,None
283,binance,BCH/USDT,2018-07-26 14:18:02.863,6611674,1.72602,2018-07-26 14:18:07.141,834.300000,sell,None
114,binance,BTC/USDT,2018-07-26 14:18:07.372,52353266,0.00890,2018-07-26 14:18:07.589,8203.010000,sell,None
113,binance,BTG/BTC,2018-07-26 14:18:07.011,3237288,0.05000,2018-07-26 14:18:09.197,0.003692,buy,None
259,binance,BTG/ETH,2018-07-26 08:55:22.790,508894,1.18000,2018-07-26 14:08:35.763,0.064787,buy,None
288,binance,DASH/BTC,2018-07-26 14:17:01.301,3629470,2.08500,2018-07-26 14:17:19.762,0.030111,buy,None


In [93]:
# calculating timestamps for _cache parameter in Database and Markets classes
ts = [int(x/1e6) for x in df.ts.values.astype(np.uint64)]

In [244]:
ex_pair_ts = [(ex, pair, int(ts/1e6)) for ex, pair, ts in zip(df.exchange.values, df.pair.values, df.ts.values.astype(np.uint64))]

In [241]:
_cache = {}
for ex, p, ts in ex_pair_ts:
    if ex not in _cache:
        _cache[ex] = {}
    if p not in _cache[ex]:
        _cache[ex][p] = {}
    _cache[ex][p] = ts

In [242]:
{_cache[ex][p] = ts}

{'binance': {'ADA/BTC': 1532614673964,
  'ADA/ETH': 1532614670734,
  'ADA/USDT': 1532614681744,
  'BCH/BTC': 1532614684607,
  'BCH/ETH': 1532614597593,
  'BCH/USDT': 1532614682863,
  'BTC/USDT': 1532614687372,
  'BTG/BTC': 1532614687011,
  'BTG/ETH': 1532595322790,
  'DASH/BTC': 1532614621301,
  'DASH/ETH': 1532614640687,
  'EOS/BTC': 1532614682881,
  'EOS/ETH': 1532614679757,
  'EOS/USDT': 1532614691797,
  'ETC/BTC': 1532614685993,
  'ETC/ETH': 1532614673716,
  'ETC/USDT': 1532614697784,
  'ETH/BTC': 1532614698771,
  'ETH/USDT': 1532614699396,
  'ICX/BTC': 1532614680297,
  'ICX/ETH': 1532614637127,
  'ICX/USDT': 1532614669535,
  'LTC/BTC': 1532614651725,
  'LTC/ETH': 1532614645787,
  'LTC/USDT': 1532614648790,
  'NEO/BTC': 1532614655991,
  'NEO/ETH': 1532614633781,
  'NEO/USDT': 1532614652989,
  'OMG/BTC': 1532614642529,
  'OMG/ETH': 1532614609188,
  'SC/BTC': 1532614648259,
  'SC/ETH': 1532614432046,
  'TRX/BTC': 1532614662327,
  'TRX/ETH': 1532614632425,
  'TRX/USDT': 1532614656811,

# Fetch historic data from exchanges

In [11]:
exchanges = df_exchangs.id.tolist()
exchanges

['kraken',
 'poloniex',
 'yobit',
 'binance',
 'okex',
 'bittrex',
 'exmo',
 'hitbtc2',
 'cryptopia']

In [172]:
pair = "ETH/USD"
pair_alt = "ETH/USDT" # alternative representation
history = {}

# loading pairs from exchanges
for exchange in exchanges:
    history[exchange] = markets.fetch_trades(exchange, pair, limit=100)
    if history[exchange] == None:
        history[exchange] = markets.fetch_trades(exchange, pair_alt, limit=100)
    elif history[exchange] == None:
        del(history[exchange])

poloniex No market symbol ETH/USD
binance No market symbol ETH/USD
hitbtc2 No market symbol ETH/USD
cryptopia No market symbol ETH/USD


## Kraken and Cryptopia has no IDs so we need to generate them by ourselves:

In [173]:
infos = {}
for ex in exchanges:
    infos[ex] = history[ex][0]['id']
    
pprint(infos)

{'binance': '28856849',
 'bittrex': '1408060',
 'cryptopia': None,
 'exmo': '65012105',
 'hitbtc2': '335014018',
 'kraken': None,
 'okex': '1150815922193419',
 'poloniex': '10123268',
 'yobit': '201211101'}


## Check whether an Exchange have full info on transaction side and type

In [174]:
for ex in exchanges:
    print("\n============\n  " + ex + "\n============")
    pprint(history[ex][0])


  kraken
{'amount': 0.06050148,
 'cost': 28.0315457136,
 'datetime': '2018-07-22T19:57:02.139Z',
 'fee': None,
 'id': None,
 'info': ['463.32000', '0.06050148', 1532289422.139, 'b', 'l', ''],
 'order': None,
 'price': 463.32,
 'side': 'buy',
 'symbol': 'ETH/USD',
 'timestamp': 1532289422139,
 'type': 'limit'}

  poloniex
{'amount': 0.50178427,
 'cost': 233.85687475,
 'datetime': '2018-07-22T20:38:44.000Z',
 'fee': None,
 'id': '10123268',
 'info': {'amount': '0.50178427',
          'date': '2018-07-22 20:38:44',
          'globalTradeID': 383626542,
          'rate': '466.05062920',
          'total': '233.85687475',
          'tradeID': 10123268,
          'type': 'sell'},
 'order': None,
 'price': 466.0506292,
 'side': 'sell',
 'symbol': 'ETH/USDT',
 'timestamp': 1532291924000,
 'type': 'limit'}

  yobit
{'amount': 0.02420813,
 'datetime': '2018-07-22T20:03:39.000Z',
 'fee': {'cost': 0.022375898829684,
         'currency': 'USD',
         'rate': 0.002,
         'type': 'taker'},
 '

## Compare prices

In [180]:
prices = [{'exchange':ex, 'price':history[ex][-1]['price']} for ex in exchanges]

In [181]:
df = pd.DataFrame(prices)
df

,exchange,price
0,kraken,463.760000
1,poloniex,459.297498
2,yobit,461.109978
3,binance,457.520000
4,okex,458.006000
5,bittrex,466.787000
6,exmo,459.392000
7,hitbtc2,459.120000
8,cryptopia,466.854417


In [182]:
# maximum spread
df_minmax = df.iloc[[df.price.idxmin(),df.price.idxmax()]]
df_minmax

,exchange,price
3,binance,457.520000
8,cryptopia,466.854417


In [191]:
# calculate diff and percentage
print(f"Difference between min and max = "
      f"{df_minmax.iloc[1].price - df_minmax.iloc[0].price} {pair.split('/')[1]}"
      f" or {round((df_minmax.iloc[1].price / df_minmax.iloc[0].price - 1)*100, 2)}%")

Difference between min and max = 9.33441732 USD or 2.04%
